# Cinetel Data Cleaning

Cinetel data scraped daily from their boxoffice published everyday at this [link](https://www.cinetel.it/pages/boxoffice.php?edperiodo=aWVyaQ==).

## Reading In Files

In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv(r"E:\data_analysis_python\cinetel\cinetel_.csv")
df1.head()

,Pos.,Titolo,Prima Progr.,Nazione,Distribuzione,Incasso,Presenze,Incasso al 16/11/2023,Presenze al 16/11/2023,2023-11-17 15:31:50.351708
0,1,C'E' ANCORA DOMANI,'2023-10-26'26/10/2023,ITA,VISION DISTRIBUTION,449991.06,66700,15231636.85,2202169,2023-11-17 15:31:50.351708
1,2,HUNGER GAMES - LA BALLATA DELL'USIGNOLO E DEL ...,'2023-11-15'15/11/2023,USA,MEDUSA FILM S.P.A.,238466.31,32469,584902.45,79311,2023-11-17 15:31:50.351708
2,3,THE MARVELS,'2023-11-08'08/11/2023,USA,WALT DISNEY S.M.P. ITALIA,60914.35,8628,2260142.44,305291,2023-11-17 15:31:50.351708
3,4,THANKSGIVING,'2023-11-16'16/11/2023,USA,EAGLE PICTURES S.P.A.,41232.57,5709,41232.57,5709,2023-11-17 15:31:50.351708
4,5,THE OLD OAK,'2023-11-16'16/11/2023,FRA,LUCKY RED DISTRIB.,36605.89,5879,56540.79,8609,2023-11-17 15:31:50.351708


---

## Cleaning the dataframe

To clean the dataframe I decided to drop duplicate values and columns that aren't relevant to my analysis, such as `df1['Incasso al 16/11/2023']` and `df1['Presenze al 16/11/2023]`. I then decided to rename the existing columns using the snake_case rule in order to make the analysis easier.

In [3]:
df1 = df1.drop_duplicates()

In [4]:
# df1 = df1.drop(columns= ['Incasso al 16/11/2023', 'Presenze al 16/11/2023'])

# at first i thought these columns weren't relevant
# making visualizations, i realized i need the data to have accurate insights on movies that started their screenings before i started to scrape the data

In [5]:
df1 = df1.rename(columns = {'Pos.':'daily_rank', 'Titolo' : 'title', 'Prima Progr.' : 'first_screening_date', 'Nazione' : 'nation', 'Distribuzione' : 'distribution', 'Incasso' : 'daily_takings', 'Presenze' : 'daily_attendance','Incasso al 16/11/2023' : 'total_takings',  'Presenze al 16/11/2023' : 'total_attendance', '2023-11-17 15:31:50.351708' : 'date_pulled'})

In [6]:
# dropping the extra 'header row' that is imported everyday when the .csv file gets updated
df1 = df1[df1['daily_rank'].str.contains('Pos.')==False]

In [7]:
# making the 'title' and 'distribution' entries consistent by appling the title format
df1['title'] = df1['title'].apply(lambda x: x.title())
df1['distribution'] = df1['distribution'].apply(lambda x: x.title())

In [8]:
# cleaning the 'first_screening_date' entries
df1['first_screening_date'] = df1['first_screening_date'].str[12:]

In [9]:
# cleaning the column showing the date the data was pulled from the site
df1['date_pulled'] = pd.to_datetime(df1['date_pulled'])
df1['date_pulled'] = pd.to_datetime(df1['date_pulled']).dt.date

#extra tries:
#df1['date_pulled'] = df1['date_pulled'].str[:10]
#df1['date_pulled'] = df1['date_pulled'].str.replace('[^a-zA-Z0-9]', '', regex = True)
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: str(x))
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: x[0:2] + '/' + x[2:4] + '/' + x[4:8])

In [10]:
from datetime import timedelta, datetime

In [11]:
# formatting
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date'], format='%d/%m/%Y')
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date']).dt.date

In [12]:
df1['daily_takings'] = df1['daily_takings'].astype(float)
df1['daily_attendance'] = df1['daily_attendance'].astype(int)
df1['daily_rank'] = df1['daily_rank'].astype(int)
df1['total_takings'] = df1['total_takings'].astype(float)
df1['total_attendance'] = df1['total_attendance'].astype(int)

---

## Data manipulation

I decided to create new columns to get information I may need later on during the analysis.

In [13]:
# calculating the number of days of screenings for each movie 
df1['screening_days'] = datetime.today().date() - df1['first_screening_date']
df1['screening_days'] = df1['screening_days'].map(lambda x: str(x)[:-14])

In [14]:
# creating a column to show the date the 'daily_takings' and 'daily_attendace' refer to
df1['date'] = df1['date_pulled'] - timedelta(days=1)
df1['date'] = pd.to_datetime(df1['date'])

In [15]:
# creating a column showing the day of the week. it'll be useful for further analysis
df1['day_of_week'] = df1['date'].dt.day_name()

In [16]:
df1['avg_ticket_price'] = df1['daily_takings'] / df1['daily_attendance']

In [17]:
pd.set_option('display.max_rows', 8)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df1

,daily_rank,title,first_screening_date,nation,distribution,daily_takings,daily_attendance,total_takings,total_attendance,date_pulled,screening_days,date,day_of_week,avg_ticket_price
0,1,C'E' Ancora Domani,2023-10-26,ITA,Vision Distribution,449991.06,66700,15231636.85,2202169,2023-11-17,36,2023-11-16,Thursday,6.75
1,2,Hunger Games - La Ballata Dell'Usignolo E Del ...,2023-11-15,USA,Medusa Film S.P.A.,238466.31,32469,584902.45,79311,2023-11-17,16,2023-11-16,Thursday,7.34
2,3,The Marvels,2023-11-08,USA,Walt Disney S.M.P. Italia,60914.35,8628,2260142.44,305291,2023-11-17,23,2023-11-16,Thursday,7.06
3,4,Thanksgiving,2023-11-16,USA,Eagle Pictures S.P.A.,41232.57,5709,41232.57,5709,2023-11-17,15,2023-11-16,Thursday,7.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,7,La Guerra Dei Nonni,2023-11-30,ITA,Medusa Film S.P.A.,20429.01,3256,20429.01,3256,2023-12-01,,2023-11-30,Thursday,6.27
161,8,Palazzina Laf,2023-11-30,ITA,Bim Distrib. S.R.L.,16126.29,2613,35091.09,5442,2023-12-01,,2023-11-30,Thursday,6.17
162,9,Il Silenzio Della Vendetta (Silent Night),2023-11-30,USA,Plaion Pictures,15751.12,2234,15751.12,2234,2023-12-01,,2023-11-30,Thursday,7.05
163,10,The Old Oak,2023-11-16,GBR,Lucky Red Distrib.,9844.48,1577,726845.84,112847,2023-12-01,15,2023-11-30,Thursday,6.24


---

## Export as .csv

In [18]:
import os

In [19]:
clean_file_name = 'cinetel_clean.csv'
clean_file_path = os.path.join(os.getcwd(), clean_file_name)

In [20]:
df1.to_csv(clean_file_path, index=False)
print(f"Il DataFrame è stato esportato con successo in {clean_file_path}.")

Il DataFrame è stato esportato con successo in e:\data_analysis_python\cinetel\cinetel_clean.csv.


---

## Data Exploration

In [21]:
import seaborn as sns
import matplotlib.pyplot as plt

In [22]:
pd.set_option('display.max_rows', 8)

In [23]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150 entries, 0 to 163
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   daily_rank            150 non-null    int32         
 1   title                 150 non-null    object        
 2   first_screening_date  150 non-null    object        
 3   nation                150 non-null    object        
 4   distribution          150 non-null    object        
 5   daily_takings         150 non-null    float64       
 6   daily_attendance      150 non-null    int32         
 7   total_takings         150 non-null    float64       
 8   total_attendance      150 non-null    int32         
 9   date_pulled           150 non-null    object        
 10  screening_days        150 non-null    object        
 11  date                  150 non-null    datetime64[ns]
 12  day_of_week           150 non-null    object        
 13  avg_ticket_price      150

In [24]:
df1.describe()

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,date,avg_ticket_price
count,150.00,150.00,150.00,150.00,150.00,150,150.00
mean,5.50,161633.00,23017.82,3711602.41,531273.72,2023-11-23 00:00:00,6.85
min,1.00,8904.68,1216.00,11652.78,1470.00,2023-11-16 00:00:00,4.22
25%,3.00,19676.50,3329.00,370950.55,51240.50,2023-11-19 00:00:00,6.24
50%,5.50,44675.18,6743.00,1862971.04,272123.50,2023-11-23 00:00:00,6.78
75%,8.00,185675.48,26367.00,3415717.60,497284.00,2023-11-27 00:00:00,7.33
max,10.00,1544231.00,211764.00,25413491.17,3721888.00,2023-11-30 00:00:00,10.94
std,2.88,266673.50,37239.35,6098210.03,886284.55,NaN,1.06


In [25]:
df1.isnull().sum()

daily_rank              0
title                   0
first_screening_date    0
nation                  0
                       ..
screening_days          0
date                    0
day_of_week             0
avg_ticket_price        0
Length: 14, dtype: int64

In [26]:
df1.nunique()

daily_rank               10
title                    24
first_screening_date     13
nation                    5
                       ... 
screening_days           13
date                     15
day_of_week               7
avg_ticket_price        150
Length: 14, dtype: int64

In [27]:
df1.corr(numeric_only=True)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,avg_ticket_price
daily_rank,1.00,-0.64,-0.65,-0.50,-0.49,-0.14
daily_takings,-0.64,1.00,0.99,0.64,0.63,0.15
daily_attendance,-0.65,0.99,1.00,0.69,0.69,0.10
total_takings,-0.50,0.64,0.69,1.00,1.00,-0.14
total_attendance,-0.49,0.63,0.69,1.00,1.00,-0.16
avg_ticket_price,-0.14,0.15,0.10,-0.14,-0.16,1.00


In [28]:
df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,avg_ticket_price
title,,,,,,
C'E' Ancora Domani,1.13,698220.30,104133.47,21078477.86,3058017.80,6.52
Io Capitano,8.00,16027.86,3670.00,4226997.42,735242.60,4.38
Me Contro Te Il Film – Vacanze In Transilvania,10.00,37520.80,5383.00,4551425.30,674907.00,6.97
Killers Of The Flower Moon,9.67,23930.22,3229.33,5007558.46,673423.33,7.20
...,...,...,...,...,...,...
Diabolik - Chi Sei?,6.00,25605.75,3963.00,25605.75,3963.00,6.46
La Guerra Dei Nonni,7.00,20429.01,3256.00,20429.01,3256.00,6.27
Casanova Operapop - Il Film,9.00,12093.47,1256.33,25578.93,2715.33,9.62
Il Silenzio Della Vendetta (Silent Night),9.00,15751.12,2234.00,15751.12,2234.00,7.05


In [29]:
df2 = df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)
df2

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,avg_ticket_price
title,,,,,,
C'E' Ancora Domani,1.13,698220.30,104133.47,21078477.86,3058017.80,6.52
Io Capitano,8.00,16027.86,3670.00,4226997.42,735242.60,4.38
Me Contro Te Il Film – Vacanze In Transilvania,10.00,37520.80,5383.00,4551425.30,674907.00,6.97
Killers Of The Flower Moon,9.67,23930.22,3229.33,5007558.46,673423.33,7.20
...,...,...,...,...,...,...
Diabolik - Chi Sei?,6.00,25605.75,3963.00,25605.75,3963.00,6.46
La Guerra Dei Nonni,7.00,20429.01,3256.00,20429.01,3256.00,6.27
Casanova Operapop - Il Film,9.00,12093.47,1256.33,25578.93,2715.33,9.62
Il Silenzio Della Vendetta (Silent Night),9.00,15751.12,2234.00,15751.12,2234.00,7.05
